# Multi-Step Tool Use

## 도구 설정하기

In [1]:
import os
import json
from typing import List, Dict
from openai import OpenAI
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

### 1단계: 도구 생성하기

이제 두 가지 도구를 만들어 보자:

- 주어진 날짜를 기준으로 기존 캘린더 이벤트를 나열하는 `list_calendar_events` 함수. 단순화를 위해 실제 캘린더에 연결하는 대신 모의 이벤트 기록을 포함할 것이다.
- 제공된 날짜, 시간, 기간을 기준으로 새 캘린더 이벤트를 생성하는 `Calendar` 함수. 역시 단순화를 위해 실제 데이터베이스를 변경하는 대신 간단한 성공 메시지를 반환할 것이다.

In [2]:
def list_calendar_events(date: str) -> str:
    """주어진 날짜의 기존 이벤트 목록을 반환한다."""
    print(f"[도구 호출] list_calendar_events 호출됨 (날짜: {date})")
    events = [{"start": "8:00", "end": "8:59"}, {"start": "9:00", "end": "9:59"}, {"start": "11:00", "end": "11:59"},{"start": "12:00", "end": "12:59"}]
    return json.dumps({"existing_events": events})

def create_calendar_event(date: str, time: str, duration: int) -> str:
    """새로운 캘린더 이벤트를 생성한다."""
    print(f"[도구 호출] create_calendar_event 호출됨 (날짜: {date}, 시간: {time}, 기간: {duration}시간)")
    return json.dumps({
        "is_success": True,
        "message": f"{date} {time}에 {duration}시간 길이의 이벤트가 생성되었습니다."
    })

available_tools = {
    "list_calendar_events": list_calendar_events,
    "create_calendar_event": create_calendar_event
}

### 2단계: 도구 스키마 정의하기

다음으로 두 도구에 대한 스키마를 정의한다.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_calendar_events",
            "description": "지정된 날짜의 기존 이벤트 목록을 반환하며, 각 이벤트의 시작 및 종료 시간을 포함한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "이벤트를 나열할 날짜, YYYY-MM-DD 형식이다."
                    }
                },
                "required": ["date"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "지정된 시간과 날짜에 지정된 기간의 새 캘린더 이벤트를 생성한다. 기존 이벤트와 동일한 시간에 새 이벤트를 생성할 수 없다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "이벤트가 시작되는 날짜, YYYY-MM-DD 형식이다."
                    },
                    "time": {
                        "type": "string",
                        "description": "이벤트 시간, HH:MM 형식의 24시간 표기법을 사용한다."
                    },
                    "duration": {
                        "type": "integer",
                        "description": "이벤트가 지속되는 시간(시간 단위).",
                        "minimum": 1
                    }
                },
                "required": ["date", "time", "duration"],
                "additionalProperties": False
            }
        }
    }
]

### 3단계: 사용자 지정 시스템 프롬프트 생성하기 

In [4]:
# 현재 날짜를 동적으로 설정한다.
from datetime import datetime
today = datetime.now().strftime('%Y-%m-%d')

system_prompt = f"""## 임무 및 컨텍스트
당신은 사람들이 캘린더에 이벤트를 예약하는 것을 돕는 캘린더 어시스턴트입니다. 새 이벤트가 기존 이벤트와 겹치지 않도록 해야 합니다.
오늘은 {today}입니다.
"""

## 다중 단계 도구 사용

- 사용자 메시지를 받는다 (1단계)
- 루프를 시작하여, 모델이 최종 답변을 내놓을 때까지 다음을 반복한다:
  - Chat API를 호출하여 다음 행동(도구 호출 또는 최종 답변)을 결정한다 (2단계)
  - 응답에 도구 호출이 있으면 실행하고 결과를 얻는다 (3단계)
  - 도구 호출이 없으면 루프를 종료하고 최종 응답을 생성한다 (4단계)

In [5]:
MODEL = "gpt-4o-mini"

def run_assistant(message: str, chat_history: List[Dict] = None) -> List[Dict]:
    print(f"질문:\n{message}")
    print("="*50)
    
    # 1단계: 대화 기록 초기화 및 사용자 메시지 추가
    if chat_history is None:
        chat_history = [{"role": "system", "content": system_prompt}]
    chat_history.append({"role": "user", "content": message})
    
    # 2단계 & 3단계: 다중 단계 추론 루프
    while True:
        response = client.chat.completions.create(
            model=MODEL,
            messages=chat_history,
            tools=tools,
            tool_choice="auto"
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        
        # 어시스턴트의 응답(도구 호출 포함 가능)을 기록에 추가
        chat_history.append(response_message)
        
        if not tool_calls:
            # 도구 호출이 없으면 루프를 중단하고 최종 답변으로 넘어간다.
            break
        
        print("도구 호출 결정:")
        for call in tool_calls:
            print(f"- 도구: {call.function.name} | 매개변수: {call.function.arguments}")
        print("="*50)
        
        # 도구 호출 실행
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_tools[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            
            # 도구 실행 결과를 대화 기록에 추가
            chat_history.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
            
    # 4단계: 최종 응답 출력
    final_text = response_message.content
    print("최종 응답:")
    print(final_text)
    print("="*50)
    
    return chat_history

In [6]:
chat_history_single = run_assistant("오늘 회의가 몇 개나 있나요?")

질문:
오늘 회의가 몇 개나 있나요?
도구 호출 결정:
- 도구: list_calendar_events | 매개변수: {"date":"2025-08-23"}
[도구 호출] list_calendar_events 호출됨 (날짜: 2025-08-23)
최종 응답:
오늘은 총 4개의 회의가 있습니다. 각 회의의 시간은 다음과 같습니다:

1. 08:00 - 08:59
2. 09:00 - 09:59
3. 11:00 - 11:59
4. 12:00 - 12:59


In [7]:
chat_history_multi = run_assistant(
    "오전 10시 이후 첫 번째 가능한 빈 슬롯에 1시간짜리 약속을 만들어주세요."
)

질문:
오전 10시 이후 첫 번째 가능한 빈 슬롯에 1시간짜리 약속을 만들어주세요.
도구 호출 결정:
- 도구: list_calendar_events | 매개변수: {"date":"2025-08-23"}
[도구 호출] list_calendar_events 호출됨 (날짜: 2025-08-23)
도구 호출 결정:
- 도구: create_calendar_event | 매개변수: {"date":"2025-08-23","time":"10:00","duration":1}
[도구 호출] create_calendar_event 호출됨 (날짜: 2025-08-23, 시간: 10:00, 기간: 1시간)
최종 응답:
2025년 8월 23일 오전 10시에 1시간짜리 약속이 성공적으로 예약되었습니다.


In [8]:
import pprint

pprint.pprint(chat_history_multi)

[{'content': '## 임무 및 컨텍스트\n'
             '당신은 사람들이 캘린더에 이벤트를 예약하는 것을 돕는 캘린더 어시스턴트입니다. 새 이벤트가 기존 이벤트와 겹치지 '
             '않도록 해야 합니다.\n'
             '오늘은 2025-08-23입니다.\n',
  'role': 'system'},
 {'content': '오전 10시 이후 첫 번째 가능한 빈 슬롯에 1시간짜리 약속을 만들어주세요.', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_otULwYcz5qxHMgHS9caGVpk5', function=Function(arguments='{"date":"2025-08-23"}', name='list_calendar_events'), type='function')]),
 {'content': '{"existing_events": [{"start": "8:00", "end": "8:59"}, {"start": '
             '"9:00", "end": "9:59"}, {"start": "11:00", "end": "11:59"}, '
             '{"start": "12:00", "end": "12:59"}]}',
  'name': 'list_calendar_events',
  'role': 'tool',
  'tool_call_id': 'call_otULwYcz5qxHMgHS9caGVpk5'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None,

## 다중 단계, 병렬 도구 사용

In [9]:
chat_history_parallel = run_assistant(
    "오전 9시부터 오후 5시 사이의 가능한 시간에 1시간짜리 약속 두 개를 예약해주세요."
)

질문:
오전 9시부터 오후 5시 사이의 가능한 시간에 1시간짜리 약속 두 개를 예약해주세요.
도구 호출 결정:
- 도구: list_calendar_events | 매개변수: {"date":"2025-08-23"}
[도구 호출] list_calendar_events 호출됨 (날짜: 2025-08-23)
도구 호출 결정:
- 도구: create_calendar_event | 매개변수: {"date": "2025-08-23", "time": "10:00", "duration": 1}
- 도구: create_calendar_event | 매개변수: {"date": "2025-08-23", "time": "13:00", "duration": 1}
[도구 호출] create_calendar_event 호출됨 (날짜: 2025-08-23, 시간: 10:00, 기간: 1시간)
[도구 호출] create_calendar_event 호출됨 (날짜: 2025-08-23, 시간: 13:00, 기간: 1시간)
최종 응답:
2025년 8월 23일에 다음과 같이 1시간짜리 약속 두 개가 성공적으로 예약되었습니다:

1. 오전 10시부터 오전 11시까지
2. 오후 1시부터 오후 2시까지

다른 요청이 필요하시면 말씀해 주세요!


## 상태 관리 (메모리)

다중 단계 도구 사용도 같은 방식으로 작동한다. 각 턴의 채팅 기록은 다음 메시지로 구성되며, 새로운 턴마다 누적될 것이다.

- `user` 메시지
- `assistant` 메시지 (도구 호출 목록 포함)
- `tool` 메시지 (도구 결과 목록 포함)
- 최종 `assistant` 메시지 (사용자에 대한 최종 응답)